In [53]:
import numpy as np
import pandas as pd
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

import seaborn as sns
import matplotlib.pyplot as plt
import os
from datetime import date
import datetime

# Generating the training data for the Heat and Diffusion Model

In [54]:
data_dir = "./LakePIAB/"
depth_steps = 25

print(os.getcwd())

/home/robert/Projects/LakePIAB/MCL/01_data_processing


In [55]:
meterological_data_df = pd.read_csv("./../../output/py_meteorology_input.csv")
meterological_data_df = meterological_data_df[1:] # considering everything from 2nd time step

num_time_steps = meterological_data_df.shape[0]
depth_list = np.array(list(range(1, depth_steps+1)) * num_time_steps)
depth_df = pd.DataFrame(data={'depth':depth_list})

#repeating the dataframe depth_steps number of times
meterological_data_df = pd.DataFrame(np.repeat(meterological_data_df.values, depth_steps, axis=0), columns=meterological_data_df.columns)
meterological_data_df = pd.concat([depth_df, meterological_data_df], ignore_index=False, axis=1)
meterological_data_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,icemovAvg,density_snow,ice_prior,snow_prior,snowice_prior,rho_snow_prior,IceSnowAttCoeff_prior,iceFlag_prior,dt_iceon_avg_prior,icemovAvg_prior
0,1,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,5.940999,250.0,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.9704
1,2,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,5.940999,250.0,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.9704
2,3,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,5.940999,250.0,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.9704
3,4,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,5.940999,250.0,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.9704
4,5,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,5.940999,250.0,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.9704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437970,21,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,-0.119937,250.0,0.245236,0.022017,0.024323,250.0,0.77808,1.0,0.8,-0.117621
437971,22,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,-0.119937,250.0,0.245236,0.022017,0.024323,250.0,0.77808,1.0,0.8,-0.117621
437972,23,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,-0.119937,250.0,0.245236,0.022017,0.024323,250.0,0.77808,1.0,0.8,-0.117621
437973,24,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,-0.119937,250.0,0.245236,0.022017,0.024323,250.0,0.77808,1.0,0.8,-0.117621


In [56]:
input_temp_df = pd.read_csv("./../../output/py_temp_total05.csv")

flattened_inp_temp = input_temp_df.iloc[:-1,1:].to_numpy().flatten() #this iloc is to remove the time column
flattened_out_temp = input_temp_df.iloc[1:,1:].to_numpy().flatten() #this iloc is to remove the time column
time_stamp = input_temp_df['time'][1:].repeat(depth_steps).values
data = {'time':time_stamp, 'input_temp':flattened_inp_temp, 'temp_total05':flattened_out_temp, 'depth':depth_list}
input_temp_df = pd.DataFrame(data=data)
input_temp_df 

,time,input_temp,temp_total05,depth
0,2006-01-01 02:00:00,5.431685,5.405889,1
1,2006-01-01 02:00:00,5.464454,5.431945,2
2,2006-01-01 02:00:00,5.481707,5.453813,3
3,2006-01-01 02:00:00,5.490656,5.453813,4
4,2006-01-01 02:00:00,5.495258,5.487071,5
...,...,...,...,...
437970,2008-01-01 00:00:00,2.446856,2.447024,21
437971,2008-01-01 00:00:00,2.883013,2.883147,22
437972,2008-01-01 00:00:00,3.395527,3.395505,23
437973,2008-01-01 00:00:00,3.818358,3.818301,24


In [57]:
depth_list.shape

(437975,)

In [58]:
final_df = meterological_data_df.merge(input_temp_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,ice_prior,snow_prior,snowice_prior,rho_snow_prior,IceSnowAttCoeff_prior,iceFlag_prior,dt_iceon_avg_prior,icemovAvg_prior,input_temp,temp_total05
0,1,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.9704,5.431685,5.405889
1,2,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.9704,5.464454,5.431945
2,3,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.9704,5.481707,5.453813
3,4,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.9704,5.490656,5.453813
4,5,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.9704,5.495258,5.487071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437970,21,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,0.245236,0.022017,0.024323,250.0,0.77808,1.0,0.8,-0.117621,2.446856,2.447024
437971,22,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,0.245236,0.022017,0.024323,250.0,0.77808,1.0,0.8,-0.117621,2.883013,2.883147
437972,23,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,0.245236,0.022017,0.024323,250.0,0.77808,1.0,0.8,-0.117621,3.395527,3.395505
437973,24,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,0.245236,0.022017,0.024323,250.0,0.77808,1.0,0.8,-0.117621,3.818358,3.818301


In [59]:
buoyancy_data_df = pd.read_csv("./../../output/py_buoyancy.csv")
#Removing Nans
#buoyancy_data_df['n2S-2_1'] = buoyancy_data_df['n2S-2_2']
#buoyancy_data_df['n2S-2_25'] = buoyancy_data_df['n2S-2_24']
buoyancy_data_df['24'] = buoyancy_data_df['23']

buoyancy_data_df = buoyancy_data_df[1:] # considering everything from 2nd time step

flattened_buoy = buoyancy_data_df.iloc[:,1:].to_numpy().flatten()
time_stamp = buoyancy_data_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'buoyancy':flattened_buoy, 'depth':depth_list}
buoyancy_data_df = pd.DataFrame(data=data)

buoyancy_data_df

,time,buoyancy,depth
0,2006-01-01 02:00:00,-7.374831e-06,1
1,2006-01-01 02:00:00,-3.947638e-06,2
2,2006-01-01 02:00:00,-2.065018e-06,3
3,2006-01-01 02:00:00,-1.066681e-06,4
4,2006-01-01 02:00:00,-5.419952e-07,5
...,...,...,...
437970,2008-01-01 00:00:00,9.207523e-05,21
437971,2008-01-01 00:00:00,6.871447e-05,22
437972,2008-01-01 00:00:00,2.503794e-05,23
437973,2008-01-01 00:00:00,-1.369672e-06,24


In [60]:
final_df = final_df.merge(buoyancy_data_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,snow_prior,snowice_prior,rho_snow_prior,IceSnowAttCoeff_prior,iceFlag_prior,dt_iceon_avg_prior,icemovAvg_prior,input_temp,temp_total05,buoyancy
0,1,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,0.0,0.0,250.0,1.0,0.0,0.8,5.9704,5.431685,5.405889,-7.374831e-06
1,2,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,0.0,0.0,250.0,1.0,0.0,0.8,5.9704,5.464454,5.431945,-3.947638e-06
2,3,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,0.0,0.0,250.0,1.0,0.0,0.8,5.9704,5.481707,5.453813,-2.065018e-06
3,4,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,0.0,0.0,250.0,1.0,0.0,0.8,5.9704,5.490656,5.453813,-1.066681e-06
4,5,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,0.0,0.0,250.0,1.0,0.0,0.8,5.9704,5.495258,5.487071,-5.419952e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437970,21,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,0.022017,0.024323,250.0,0.77808,1.0,0.8,-0.117621,2.446856,2.447024,9.207523e-05
437971,22,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,0.022017,0.024323,250.0,0.77808,1.0,0.8,-0.117621,2.883013,2.883147,6.871447e-05
437972,23,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,0.022017,0.024323,250.0,0.77808,1.0,0.8,-0.117621,3.395527,3.395505,2.503794e-05
437973,24,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,0.022017,0.024323,250.0,0.77808,1.0,0.8,-0.117621,3.818358,3.818301,-1.369672e-06


In [61]:
out_diffusivity_df = pd.read_csv("./../../output/py_diff.csv")

out_diffusivity_df = out_diffusivity_df[1:] # considering everything from 2nd time step

flattened_diff = out_diffusivity_df.iloc[:,1:].to_numpy().flatten()
time_stamp = out_diffusivity_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'diffusivity':flattened_diff, 'depth':depth_list}
out_diffusivity_df = pd.DataFrame(data=data)

out_diffusivity_df

,time,diffusivity,depth
0,2006-01-01 02:00:00,1.859530e-05,1
1,2006-01-01 02:00:00,1.859530e-05,2
2,2006-01-01 02:00:00,1.859530e-05,3
3,2006-01-01 02:00:00,1.859530e-05,4
4,2006-01-01 02:00:00,1.859530e-05,5
...,...,...,...
437970,2008-01-01 00:00:00,2.875938e-07,21
437971,2008-01-01 00:00:00,3.179400e-07,22
437972,2008-01-01 00:00:00,3.179400e-07,23
437973,2008-01-01 00:00:00,3.179400e-07,24


In [62]:
final_df = final_df.merge(out_diffusivity_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,snowice_prior,rho_snow_prior,IceSnowAttCoeff_prior,iceFlag_prior,dt_iceon_avg_prior,icemovAvg_prior,input_temp,temp_total05,buoyancy,diffusivity
0,1,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,0.0,250.0,1.0,0.0,0.8,5.9704,5.431685,5.405889,-7.374831e-06,1.859530e-05
1,2,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,0.0,250.0,1.0,0.0,0.8,5.9704,5.464454,5.431945,-3.947638e-06,1.859530e-05
2,3,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,0.0,250.0,1.0,0.0,0.8,5.9704,5.481707,5.453813,-2.065018e-06,1.859530e-05
3,4,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,0.0,250.0,1.0,0.0,0.8,5.9704,5.490656,5.453813,-1.066681e-06,1.859530e-05
4,5,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,0.0,250.0,1.0,0.0,0.8,5.9704,5.495258,5.487071,-5.419952e-07,1.859530e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437970,21,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,0.024323,250.0,0.77808,1.0,0.8,-0.117621,2.446856,2.447024,9.207523e-05,2.875938e-07
437971,22,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,0.024323,250.0,0.77808,1.0,0.8,-0.117621,2.883013,2.883147,6.871447e-05,3.179400e-07
437972,23,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,0.024323,250.0,0.77808,1.0,0.8,-0.117621,3.395527,3.395505,2.503794e-05,3.179400e-07
437973,24,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,0.024323,250.0,0.77808,1.0,0.8,-0.117621,3.818358,3.818301,-1.369672e-06,3.179400e-07


In [63]:
out_temp_df = pd.read_csv("./../../output/py_temp_heat01.csv")

out_temp_df = out_temp_df[1:] # considering everything from 2nd time step

flattened_temp = out_temp_df.iloc[:,1:].to_numpy().flatten()
time_stamp = out_temp_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_heat01':flattened_temp, 'depth':depth_list}
out_temp_df = pd.DataFrame(data=data)

out_temp_df

,time,temp_heat01,depth
0,2006-01-01 02:00:00,5.313505,1
1,2006-01-01 02:00:00,5.464459,2
2,2006-01-01 02:00:00,5.481712,3
3,2006-01-01 02:00:00,5.490660,4
4,2006-01-01 02:00:00,5.495260,5
...,...,...,...
437970,2008-01-01 00:00:00,2.446891,21
437971,2008-01-01 00:00:00,2.883060,22
437972,2008-01-01 00:00:00,3.395608,23
437973,2008-01-01 00:00:00,3.818358,24


In [64]:
final_df = final_df.merge(out_temp_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,rho_snow_prior,IceSnowAttCoeff_prior,iceFlag_prior,dt_iceon_avg_prior,icemovAvg_prior,input_temp,temp_total05,buoyancy,diffusivity,temp_heat01
0,1,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,250.0,1.0,0.0,0.8,5.9704,5.431685,5.405889,-7.374831e-06,1.859530e-05,5.313505
1,2,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,250.0,1.0,0.0,0.8,5.9704,5.464454,5.431945,-3.947638e-06,1.859530e-05,5.464459
2,3,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,250.0,1.0,0.0,0.8,5.9704,5.481707,5.453813,-2.065018e-06,1.859530e-05,5.481712
3,4,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,250.0,1.0,0.0,0.8,5.9704,5.490656,5.453813,-1.066681e-06,1.859530e-05,5.490660
4,5,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,250.0,1.0,0.0,0.8,5.9704,5.495258,5.487071,-5.419952e-07,1.859530e-05,5.495260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437970,21,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,250.0,0.77808,1.0,0.8,-0.117621,2.446856,2.447024,9.207523e-05,2.875938e-07,2.446891
437971,22,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,250.0,0.77808,1.0,0.8,-0.117621,2.883013,2.883147,6.871447e-05,3.179400e-07,2.883060
437972,23,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,250.0,0.77808,1.0,0.8,-0.117621,3.395527,3.395505,2.503794e-05,3.179400e-07,3.395608
437973,24,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,250.0,0.77808,1.0,0.8,-0.117621,3.818358,3.818301,-1.369672e-06,3.179400e-07,3.818358


In [65]:
out_temp_df = pd.read_csv("./../../output/py_temp_diff02.csv")

out_temp_df = out_temp_df[1:] # considering everything from 2nd time step

flattened_temp = out_temp_df.iloc[:,1:].to_numpy().flatten()
time_stamp = out_temp_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_diff02':flattened_temp, 'depth':depth_list}
out_temp_df = pd.DataFrame(data=data)

out_temp_df

,time,temp_diff02,depth
0,2006-01-01 02:00:00,5.313505,1
1,2006-01-01 02:00:00,5.456045,2
2,2006-01-01 02:00:00,5.480917,3
3,2006-01-01 02:00:00,5.490358,4
4,2006-01-01 02:00:00,5.495106,5
...,...,...,...
437970,2008-01-01 00:00:00,2.447024,21
437971,2008-01-01 00:00:00,2.883147,22
437972,2008-01-01 00:00:00,3.395505,23
437973,2008-01-01 00:00:00,3.818301,24


In [66]:
final_df = final_df.merge(out_temp_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,IceSnowAttCoeff_prior,iceFlag_prior,dt_iceon_avg_prior,icemovAvg_prior,input_temp,temp_total05,buoyancy,diffusivity,temp_heat01,temp_diff02
0,1,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,1.0,0.0,0.8,5.9704,5.431685,5.405889,-7.374831e-06,1.859530e-05,5.313505,5.313505
1,2,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,1.0,0.0,0.8,5.9704,5.464454,5.431945,-3.947638e-06,1.859530e-05,5.464459,5.456045
2,3,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,1.0,0.0,0.8,5.9704,5.481707,5.453813,-2.065018e-06,1.859530e-05,5.481712,5.480917
3,4,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,1.0,0.0,0.8,5.9704,5.490656,5.453813,-1.066681e-06,1.859530e-05,5.490660,5.490358
4,5,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,1.0,0.0,0.8,5.9704,5.495258,5.487071,-5.419952e-07,1.859530e-05,5.495260,5.495106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437970,21,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,0.77808,1.0,0.8,-0.117621,2.446856,2.447024,9.207523e-05,2.875938e-07,2.446891,2.447024
437971,22,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,0.77808,1.0,0.8,-0.117621,2.883013,2.883147,6.871447e-05,3.179400e-07,2.883060,2.883147
437972,23,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,0.77808,1.0,0.8,-0.117621,3.395527,3.395505,2.503794e-05,3.179400e-07,3.395608,3.395505
437973,24,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,0.77808,1.0,0.8,-0.117621,3.818358,3.818301,-1.369672e-06,3.179400e-07,3.818358,3.818301


In [67]:
datetime_list =[datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S') for date in final_df['time']]
day_of_year_list = [t.timetuple().tm_yday for t in datetime_list]
time_of_day_list = [t.hour for t in datetime_list]

In [68]:
final_df['day_of_year']=day_of_year_list
final_df['time_of_day']=time_of_day_list

In [69]:
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,dt_iceon_avg_prior,icemovAvg_prior,input_temp,temp_total05,buoyancy,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day
0,1,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,0.8,5.9704,5.431685,5.405889,-7.374831e-06,1.859530e-05,5.313505,5.313505,1,2
1,2,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,0.8,5.9704,5.464454,5.431945,-3.947638e-06,1.859530e-05,5.464459,5.456045,1,2
2,3,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,0.8,5.9704,5.481707,5.453813,-2.065018e-06,1.859530e-05,5.481712,5.480917,1,2
3,4,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,0.8,5.9704,5.490656,5.453813,-1.066681e-06,1.859530e-05,5.490660,5.490358,1,2
4,5,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,0.8,5.9704,5.495258,5.487071,-5.419952e-07,1.859530e-05,5.495260,5.495106,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437970,21,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,0.8,-0.117621,2.446856,2.447024,9.207523e-05,2.875938e-07,2.446891,2.447024,1,0
437971,22,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,0.8,-0.117621,2.883013,2.883147,6.871447e-05,3.179400e-07,2.883060,2.883147,1,0
437972,23,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,0.8,-0.117621,3.395527,3.395505,2.503794e-05,3.179400e-07,3.395608,3.395505,1,0
437973,24,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,0.8,-0.117621,3.818358,3.818301,-1.369672e-06,3.179400e-07,3.818358,3.818301,1,0


In [70]:
temp_mix03_df = pd.read_csv("./../../output/py_temp_mix03.csv")

temp_mix03_df = temp_mix03_df[1:] # considering everything from 2nd time step

flattened_temp = temp_mix03_df.iloc[:,1:].to_numpy().flatten()
time_stamp = temp_mix03_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_mix03':flattened_temp, 'depth':depth_list}
temp_mix03_df = pd.DataFrame(data=data)

temp_mix03_df

,time,temp_mix03,depth
0,2006-01-01 02:00:00,5.382733,1
1,2006-01-01 02:00:00,5.430411,2
2,2006-01-01 02:00:00,5.459545,3
3,2006-01-01 02:00:00,5.476941,4
4,2006-01-01 02:00:00,5.487071,5
...,...,...,...
437970,2008-01-01 00:00:00,2.447024,21
437971,2008-01-01 00:00:00,2.883147,22
437972,2008-01-01 00:00:00,3.395505,23
437973,2008-01-01 00:00:00,3.818301,24


In [71]:
final_df = final_df.merge(temp_mix03_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,icemovAvg_prior,input_temp,temp_total05,buoyancy,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day,temp_mix03
0,1,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,5.9704,5.431685,5.405889,-7.374831e-06,1.859530e-05,5.313505,5.313505,1,2,5.382733
1,2,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,5.9704,5.464454,5.431945,-3.947638e-06,1.859530e-05,5.464459,5.456045,1,2,5.430411
2,3,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,5.9704,5.481707,5.453813,-2.065018e-06,1.859530e-05,5.481712,5.480917,1,2,5.459545
3,4,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,5.9704,5.490656,5.453813,-1.066681e-06,1.859530e-05,5.490660,5.490358,1,2,5.476941
4,5,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,5.9704,5.495258,5.487071,-5.419952e-07,1.859530e-05,5.495260,5.495106,1,2,5.487071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437970,21,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,-0.117621,2.446856,2.447024,9.207523e-05,2.875938e-07,2.446891,2.447024,1,0,2.447024
437971,22,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,-0.117621,2.883013,2.883147,6.871447e-05,3.179400e-07,2.883060,2.883147,1,0,2.883147
437972,23,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,-0.117621,3.395527,3.395505,2.503794e-05,3.179400e-07,3.395608,3.395505,1,0,3.395505
437973,24,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,-0.117621,3.818358,3.818301,-1.369672e-06,3.179400e-07,3.818358,3.818301,1,0,3.818301


In [72]:
temp_conv03_df = pd.read_csv("./../../output/py_temp_conv04.csv")

temp_conv03_df = temp_conv03_df[1:] # considering everything from 2nd time step

flattened_temp = temp_conv03_df.iloc[:,1:].to_numpy().flatten()
time_stamp = temp_conv03_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_conv04':flattened_temp, 'depth':depth_list}
temp_conv03_df = pd.DataFrame(data=data)

temp_conv03_df

,time,temp_conv04,depth
0,2006-01-01 02:00:00,5.405889,1
1,2006-01-01 02:00:00,5.431945,2
2,2006-01-01 02:00:00,5.453813,3
3,2006-01-01 02:00:00,5.453813,4
4,2006-01-01 02:00:00,5.487071,5
...,...,...,...
437970,2008-01-01 00:00:00,2.447024,21
437971,2008-01-01 00:00:00,2.883147,22
437972,2008-01-01 00:00:00,3.395505,23
437973,2008-01-01 00:00:00,3.818301,24


In [73]:
final_df = final_df.merge(temp_conv03_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,input_temp,temp_total05,buoyancy,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day,temp_mix03,temp_conv04
0,1,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,5.431685,5.405889,-7.374831e-06,1.859530e-05,5.313505,5.313505,1,2,5.382733,5.405889
1,2,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,5.464454,5.431945,-3.947638e-06,1.859530e-05,5.464459,5.456045,1,2,5.430411,5.431945
2,3,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,5.481707,5.453813,-2.065018e-06,1.859530e-05,5.481712,5.480917,1,2,5.459545,5.453813
3,4,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,5.490656,5.453813,-1.066681e-06,1.859530e-05,5.490660,5.490358,1,2,5.476941,5.453813
4,5,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,5.495258,5.487071,-5.419952e-07,1.859530e-05,5.495260,5.495106,1,2,5.487071,5.487071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437970,21,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,2.446856,2.447024,9.207523e-05,2.875938e-07,2.446891,2.447024,1,0,2.447024,2.447024
437971,22,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,2.883013,2.883147,6.871447e-05,3.179400e-07,2.883060,2.883147,1,0,2.883147,2.883147
437972,23,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,3.395527,3.395505,2.503794e-05,3.179400e-07,3.395608,3.395505,1,0,3.395505,3.395505
437973,24,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,3.818358,3.818301,-1.369672e-06,3.179400e-07,3.818358,3.818301,1,0,3.818301,3.818301


In [74]:
temp_initial00_df = pd.read_csv("./../../output/py_temp_initial00.csv")

temp_initial00_df = temp_initial00_df[1:] # considering everything from 2nd time step

flattened_temp = temp_initial00_df.iloc[:,1:].to_numpy().flatten()
time_stamp = temp_initial00_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_initial00':flattened_temp, 'depth':depth_list}
temp_initial00_df = pd.DataFrame(data=data)

temp_initial00_df

,time,temp_initial00,depth
0,2006-01-01 02:00:00,5.431685,1
1,2006-01-01 02:00:00,5.464454,2
2,2006-01-01 02:00:00,5.481707,3
3,2006-01-01 02:00:00,5.490656,4
4,2006-01-01 02:00:00,5.495258,5
...,...,...,...
437970,2008-01-01 00:00:00,2.446856,21
437971,2008-01-01 00:00:00,2.883013,22
437972,2008-01-01 00:00:00,3.395527,23
437973,2008-01-01 00:00:00,3.818358,24


In [75]:
final_df = final_df.merge(temp_initial00_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,temp_total05,buoyancy,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day,temp_mix03,temp_conv04,temp_initial00
0,1,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,5.405889,-7.374831e-06,1.859530e-05,5.313505,5.313505,1,2,5.382733,5.405889,5.431685
1,2,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,5.431945,-3.947638e-06,1.859530e-05,5.464459,5.456045,1,2,5.430411,5.431945,5.464454
2,3,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,5.453813,-2.065018e-06,1.859530e-05,5.481712,5.480917,1,2,5.459545,5.453813,5.481707
3,4,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,5.453813,-1.066681e-06,1.859530e-05,5.490660,5.490358,1,2,5.476941,5.453813,5.490656
4,5,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,5.487071,-5.419952e-07,1.859530e-05,5.495260,5.495106,1,2,5.487071,5.487071,5.495258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437970,21,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,2.447024,9.207523e-05,2.875938e-07,2.446891,2.447024,1,0,2.447024,2.447024,2.446856
437971,22,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,2.883147,6.871447e-05,3.179400e-07,2.883060,2.883147,1,0,2.883147,2.883147,2.883013
437972,23,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,3.395505,2.503794e-05,3.179400e-07,3.395608,3.395505,1,0,3.395505,3.395505,3.395527
437973,24,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,3.818301,-1.369672e-06,3.179400e-07,3.818358,3.818301,1,0,3.818301,3.818301,3.818358


In [76]:
temp_obs_df = pd.read_csv("./../../output/py_observed_temp.csv")

temp_obs_df = temp_obs_df[1:] # considering everything from 2nd time step

flattened_temp = temp_obs_df.iloc[:,1:].to_numpy().flatten()
time_stamp = temp_obs_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'obs_temp':flattened_temp, 'depth':depth_list}

print(data)
print(len(data['time']))
print(len(data['obs_temp']))
print(len(data['depth']))

temp_obs_df = pd.DataFrame(data=data)

temp_obs_df

print(flattened_temp.shape)

print(time_stamp.shape)

print(depth_list.shape)

{'time': array(['2006-01-01 02:00:00', '2006-01-01 02:00:00',
       '2006-01-01 02:00:00', ..., '2008-01-01 00:00:00',
       '2008-01-01 00:00:00', '2008-01-01 00:00:00'], dtype=object), 'obs_temp': array([-999., -999., -999., ..., -999., -999., -999.]), 'depth': array([ 1,  2,  3, ..., 23, 24, 25])}
437975
437975
437975
(437975,)
(437975,)
(437975,)


In [77]:
final_df = final_df.merge(temp_obs_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,buoyancy,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day,temp_mix03,temp_conv04,temp_initial00,obs_temp
0,1,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,-7.374831e-06,1.859530e-05,5.313505,5.313505,1,2,5.382733,5.405889,5.431685,-999.0
1,2,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,-3.947638e-06,1.859530e-05,5.464459,5.456045,1,2,5.430411,5.431945,5.464454,-999.0
2,3,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,-2.065018e-06,1.859530e-05,5.481712,5.480917,1,2,5.459545,5.453813,5.481707,-999.0
3,4,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,-1.066681e-06,1.859530e-05,5.490660,5.490358,1,2,5.476941,5.453813,5.490656,-999.0
4,5,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,-5.419952e-07,1.859530e-05,5.495260,5.495106,1,2,5.487071,5.487071,5.495258,-999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437970,21,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,9.207523e-05,2.875938e-07,2.446891,2.447024,1,0,2.447024,2.447024,2.446856,-999.0
437971,22,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,6.871447e-05,3.179400e-07,2.883060,2.883147,1,0,2.883147,2.883147,2.883013,-999.0
437972,23,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,2.503794e-05,3.179400e-07,3.395608,3.395505,1,0,3.395505,3.395505,3.395527,-999.0
437973,24,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,-1.369672e-06,3.179400e-07,3.818358,3.818301,1,0,3.818301,3.818301,3.818358,-999.0


In [78]:

#temp_obs_df2 = pd.read_csv("./../output/observed_temp.csv")
#temp_obs_df = pd.read_csv("./../output/observed_temp.csv")


#print(flattened_inp_temp_obs.shape)

#temp_obs_df2 = temp_obs_df2[:-1] # considering everything from 2nd time step

#flattened_inp_temp_obs = temp_obs_df2.iloc[:,1:].to_numpy().flatten()

#time_stamp = input_temp_df['time'][1:].repeat(depth_steps).values

#data = {'time':time_stamp, 'obs_input':flattened_inp_temp_obs, 'depth':depth_list}
#temp_obs_df2 = pd.DataFrame(data=data)

#temp_obs_df2


temp_obs_df = pd.read_csv("./../../output/py_observed_temp.csv")

flattened_inp_temp = temp_obs_df.iloc[:-1,1:].to_numpy().flatten() #this iloc is to remove the time column
print(flattened_inp_temp.shape)


time_stamp = temp_obs_df['time'][1:].repeat(depth_steps).values
print(time_stamp.shape)

print(depth_list.shape)
data = {'time':time_stamp, 'input_obs':flattened_inp_temp,'depth':depth_list}
temp_obs_df = pd.DataFrame(data=data)
temp_obs_df 

(437975,)
(437975,)
(437975,)


,time,input_obs,depth
0,2006-01-01 02:00:00,-999.0,1
1,2006-01-01 02:00:00,-999.0,2
2,2006-01-01 02:00:00,-999.0,3
3,2006-01-01 02:00:00,-999.0,4
4,2006-01-01 02:00:00,-999.0,5
...,...,...,...
437970,2008-01-01 00:00:00,-999.0,21
437971,2008-01-01 00:00:00,-999.0,22
437972,2008-01-01 00:00:00,-999.0,23
437973,2008-01-01 00:00:00,-999.0,24


In [79]:
final_df = final_df.merge(temp_obs_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day,temp_mix03,temp_conv04,temp_initial00,obs_temp,input_obs
0,1,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,1.859530e-05,5.313505,5.313505,1,2,5.382733,5.405889,5.431685,-999.0,-999.0
1,2,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,1.859530e-05,5.464459,5.456045,1,2,5.430411,5.431945,5.464454,-999.0,-999.0
2,3,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,1.859530e-05,5.481712,5.480917,1,2,5.459545,5.453813,5.481707,-999.0,-999.0
3,4,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,1.859530e-05,5.490660,5.490358,1,2,5.476941,5.453813,5.490656,-999.0,-999.0
4,5,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,1.859530e-05,5.495260,5.495106,1,2,5.487071,5.487071,5.495258,-999.0,-999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437970,21,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,2.875938e-07,2.446891,2.447024,1,0,2.447024,2.447024,2.446856,-999.0,-999.0
437971,22,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,3.179400e-07,2.883060,2.883147,1,0,2.883147,2.883147,2.883013,-999.0,-999.0
437972,23,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,3.179400e-07,3.395608,3.395505,1,0,3.395505,3.395505,3.395527,-999.0,-999.0
437973,24,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,3.179400e-07,3.818358,3.818301,1,0,3.818301,3.818301,3.818358,-999.0,-999.0


In [80]:
ice_data_df = pd.read_csv("./../../output/py_icesnow.csv")
ice_data_df = ice_data_df[1:] # considering everything from 2nd time step

num_time_steps = ice_data_df.shape[0]
depth_list = np.array(list(range(1, depth_steps+1)) * num_time_steps)
depth_df = pd.DataFrame(data={'depth':depth_list})

#repeating the dataframe depth_steps number of times
ice_data_df = pd.DataFrame(np.repeat(ice_data_df.values, depth_steps, axis=0), columns=ice_data_df.columns)
ice_data_df = pd.concat([depth_df, ice_data_df], ignore_index=False, axis=1)
print(ice_data_df)

final_df = final_df.merge(ice_data_df, how='inner', on=['time','depth'])
final_df

        depth                 time       ice      snow   snowice
0           1  2006-01-01 02:00:00       0.0       0.0       0.0
1           2  2006-01-01 02:00:00       0.0       0.0       0.0
2           3  2006-01-01 02:00:00       0.0       0.0       0.0
3           4  2006-01-01 02:00:00       0.0       0.0       0.0
4           5  2006-01-01 02:00:00       0.0       0.0       0.0
...       ...                  ...       ...       ...       ...
437970     21  2008-01-01 00:00:00  0.245353  0.022017  0.024323
437971     22  2008-01-01 00:00:00  0.245353  0.022017  0.024323
437972     23  2008-01-01 00:00:00  0.245353  0.022017  0.024323
437973     24  2008-01-01 00:00:00  0.245353  0.022017  0.024323
437974     25  2008-01-01 00:00:00  0.245353  0.022017  0.024323

[437975 rows x 5 columns]


,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,day_of_year,time_of_day,temp_mix03,temp_conv04,temp_initial00,obs_temp,input_obs,ice,snow,snowice
0,1,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,1,2,5.382733,5.405889,5.431685,-999.0,-999.0,0.0,0.0,0.0
1,2,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,1,2,5.430411,5.431945,5.464454,-999.0,-999.0,0.0,0.0,0.0
2,3,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,1,2,5.459545,5.453813,5.481707,-999.0,-999.0,0.0,0.0,0.0
3,4,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,1,2,5.476941,5.453813,5.490656,-999.0,-999.0,0.0,0.0,0.0
4,5,2006-01-01 02:00:00,-1.220007,590.289546,-29.441358,-34.843469,0.0,0.4,1.757219,0.008038,...,1,2,5.487071,5.487071,5.495258,-999.0,-999.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437970,21,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,1,0,2.447024,2.447024,2.446856,-999.0,-999.0,0.245353,0.022017,0.024323
437971,22,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,1,0,2.883147,2.883147,2.883013,-999.0,-999.0,0.245353,0.022017,0.024323
437972,23,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,1,0,3.395505,3.395505,3.395527,-999.0,-999.0,0.245353,0.022017,0.024323
437973,24,2008-01-01 00:00:00,-6.479987,512.128796,-33.797728,-52.578486,0.0,0.4,4.011884,0.030118,...,1,0,3.818301,3.818301,3.818358,-999.0,-999.0,0.245353,0.022017,0.024323


In [81]:
obs_array = final_df['obs_temp']
obs_array[obs_array == -999] = final_df['temp_total05']
print(obs_array)
final_df['obs_temp'] = obs_array

0         5.405889
1         5.431945
2         5.453813
3         5.453813
4         5.487071
            ...   
437970    2.447024
437971    2.883147
437972    3.395505
437973    3.818301
437974    4.192072
Name: obs_temp, Length: 437975, dtype: float64


/home/robert/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [82]:
final_df_null = final_df.fillna('')
print(final_df_null.head)

<bound method NDFrame.head of         depth                 time  AirTemp_degC  Longwave_Wm-2  Latent_Wm-2  \
0           1  2006-01-01 02:00:00     -1.220007     590.289546   -29.441358   
1           2  2006-01-01 02:00:00     -1.220007     590.289546   -29.441358   
2           3  2006-01-01 02:00:00     -1.220007     590.289546   -29.441358   
3           4  2006-01-01 02:00:00     -1.220007     590.289546   -29.441358   
4           5  2006-01-01 02:00:00     -1.220007     590.289546   -29.441358   
...       ...                  ...           ...            ...          ...   
437970     21  2008-01-01 00:00:00     -6.479987     512.128796   -33.797728   
437971     22  2008-01-01 00:00:00     -6.479987     512.128796   -33.797728   
437972     23  2008-01-01 00:00:00     -6.479987     512.128796   -33.797728   
437973     24  2008-01-01 00:00:00     -6.479987     512.128796   -33.797728   
437974     25  2008-01-01 00:00:00     -6.479987     512.128796   -33.797728   

        S

In [83]:
final_df_null.to_csv("./../02_training/all_data_lake_modeling_in_time.csv", index=False)

In [84]:
final_df_null.to_csv("all_data_lake_modeling_in_time.csv", index=False)